# P5 → Ocean Warming 

## Introduction
Because of the human driven intensification of the greenhouse effect, the ocean is warming. This project is about caracterising and looking at this warming. 

*Bibliography*:

- [Ocean Climate scientific sheet](https://ocean-climate.org/wp-content/uploads/2020/01/1.-The-ocean-a-heat-reservoir-scientific-fact-sheets-2019.pdf)
- [Last IPCC report on Ocean Heat Content changes](https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_Chapter_09.pdf#page=35)
- [IPCC fig 9.6](https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_Chapter_09.pdf#page=227)
- [IUCN brief on Ocean warming](https://www.iucn.org/resources/issues-briefs/ocean-warming)
- [Global Ocean Heat and Salt Content: Seasonal, Yearly, and Pentadal Fields](https://www.ncei.noaa.gov/access/global-ocean-heat-content/)
- [European Indicator](https://marine.copernicus.eu/access-data/ocean-monitoring-indicators/global-ocean-heat-content-0-2000m)

*Ideas for your project*

Below is a list of what you could do in a timely manner in your project.

You shall first compute ocean heat content (OHC) and caracterise its timeseries. Then you could compute its trend with a regression (linear or not) for the entire ocean time series and extrapolate to the future, e.g. what is the expected ocean warming for the horizon 2100?

Instead of working globally, you could also study the ocean warming locally. In this case, you can plot the local slopes of the different OHC, or temperature time series and deduce where the ocean warming is moderate and where it is strong. You could answer the question of where global warming affects most the ocean ?

The EN4 dataset to work with is a global interpolation of all available ocean in-situ observations. You could further compare CMIP6 projections or historical simulations to the EN4 reference.

## Description
Ocean Heat Content $OHC(x, y, z, t)$ is expressed in Joules an is given by:
$$OHC(x,y,z,t) = \rho \cdot C_p \cdot T(x,y,z,t) \cdot \delta V(x,y,z)$$

where $\rho$ is density and $C_p$ heat capacity:
* $\rho$ = 1035 $kg/m^{3}$
* $C_{\rho}$ =  4,186 J/(kg ⋅ K)
  
They will be considered as constant for the project.

The ocean temperature $T$ can be obtained directly from the dataset. Be careful with units.

The main problem in this project is to calculate the volume element $\delta V$ for a given latitude, longitude and depth. This volume element represents the ocean volume where temperature $T(x,y,z,t)$ is considered constant in the dataset. 
The larger the volume, the larger the error compared to the real ocean. So, we'll keep in mind that the dataset, although based on observations, remains an interpolation with a limited scientific scope, given by the grid resolution.

The P5 project requires you to compute the $OHC(x,y,z,t)$ variable and to explore its properties, e.g.: 0-700m vertical integral, zonal integral, global timeseries, 0-700 mean temperature maps.

# Getting started

## Load all the necessary libraries

In [ ]:
import os, sys, urllib, tempfile
with tempfile.TemporaryDirectory() as tmpdirname:
    sys.path.append(tmpdirname)
    repo = "https://raw.githubusercontent.com/obidam/ds2-2025/main/"
    urllib.request.urlretrieve(os.path.join(repo, "utils.py"), 
                               os.path.join(tmpdirname, "utils.py"))
    from utils import check_up_env
    ds2tools = check_up_env(with_tuto=True)

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats
import dask

from intake import open_catalog

import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
create_map = ds2tools.create_map

## Connect to the dataset

In [ ]:
catalog_url = 'https://raw.githubusercontent.com/obidam/ds2-2025/main/ds2_data_catalog.yml'
cat = open_catalog(catalog_url)
ds = cat["en4"].to_dask()
print("Size of the dataset:", ds.nbytes/1e9,"Gb")
ds

## Explore the dataset
Before moving on to computing OHC, you can make some maps of the dataset variables to familiarize with it.

In [ ]:
# Get the dataset temperature variable in deg C:
T = ds['temperature'] - 273.15  # Since unit is K

# Then select the level the closest to the surface, to get an estimate of the Sea Surface Temperarure (SST)
SST = T.sel(depth=0, method='nearest')
SST

In [ ]:
# Rough plot with xarray:
SST.isel(time=0).plot(vmin=-2, vmax=40)

In [ ]:
# A nicer plot for presentation:
fig, proj, ax = create_map()
SST.isel(time=0).plot(transform=proj, ax=ax, vmin=-2, vmax=30, cbar_kwargs={'shrink': 0.4})
ax.add_feature(cfeature.LAND, facecolor=[0.7]*3, zorder=100)

In [ ]:
# You can also use multi-dimensional indexing:
T.sel(time='2005-11', method='nearest').sel(depth=1000, method='nearest').plot(vmin=-2, vmax=10)

# Analyse OHC and temperature

## Compute volume elements
With the EN4 dataset, the grid is a regular Mercator projection, i.e. it has constant spacing in latitude and longitude in degrees. In this case, each grid point is separated by 1 degree from nearby grid points.

To account for the spherical geometry of the globe, the volume element $\delta V$ will smaller near the pole than at the equator. Since the EN4 dataset grid is regular, volume elements only depends on latitude and depth (because the vertical depth axis is not regular).

We will decompose it as:
$$\delta V (y,z) = \delta A(y) \cdot \delta z(z)$$

Although xarray is very useful for geospatial analysis, **it has no built-in understanding of geography**.

Below we show how to create variables to determine the volume element $\delta V$ and be able to compute a proper weighted sum for OHC.

### Surface elements
We'll be using the formula for the area element in spherical coordinates. The [area element for lat-lon coordinates](https://en.wikipedia.org/wiki/Spherical_coordinate_system#Integration_and_differentiation_in_spherical_coordinates) is

$$ \delta A (x,y) = R^2 \delta \phi \delta \lambda \cos(\phi) $$

where $\phi$ is latitude, $\delta \phi$ is the spacing of the points in latitude, $\delta \lambda$ is the spacing of the points in longitude, and $R$ is Earth's radius. (In this formula, $\phi$ and $\lambda$ are measured in radians.) Let's use xarray to create the weight factor.

Create a function that compute the surface element $\delta A$.

In [ ]:
def get_dA(ds):
    # Earth Radius in meters:
    R = 6.37e6  
    # we know already that the spacing of the points is one degree latitude:
    dϕ = np.deg2rad(1.)
    dλ = np.deg2rad(1.)
    # Apply formulae for area element for lat-lon coordinates:
    dA = 
    return xr.DataArray(-dA, dims='lat', name='dA', attrs={'unit': 'm2', 'long_name': 'Grid surface elements'})

ds['dA'] = get_dA(ds)
ds['dA'].plot()

### Depth elements

The vertical axis is trickier because depth levels are not regular. By considering the given depth as the center of cubes, you can divide the cubes in the middle of 2 given depths. For the first and the last cube, you can start from the surface of the ocean and end at the last depth available.

The following plot illustrate the thickness of the vertical grid depending on depth levels:

In [ ]:
ds['depth'].diff(dim='depth').plot(y='depth', yincrease=False, marker='.')
plt.xlabel("Depth cell thickness [m]")

Write a function that computes depth thickness elements $\delta z$:

In [ ]:
def get_dz(ds):
    depth = ds['depth'].values
    dz_0 = depth[0] + (depth[1]-depth[0])/2
    dz_i = (depth[1:-1]-depth[:-2])/2 + (depth[2:]-depth[1:-1])/2
    dz_N = (depth[-1]-depth[-2])/2
    dz = np.concatenate((dz_0[np.newaxis], dz_i, dz_N[np.newaxis]))
    return xr.DataArray(dz, dims='depth', name='dZ', attrs={'unit': 'm', 'long_name': 'Grid vertical thickness'})
    
ds['dZ'] = get_dz(ds)
ds['dZ'].plot()

### Volume elements

Now that you have thickness and surface elements, you can simply compute volume elements like this:

(xarray will automatically handle the dimensions broadcasting)

In [ ]:
ds['dV'] = ds['dA'] * ds['dZ']
ds['dV'].attrs = {'unit': 'm3', 'long_name': 'Grid volume elements'}
ds['dV']

## Timeseries of OHC

### Connect to a Coiled Dask cluster

But this will be a computational expensive operation, so you better connect to the Coiled Dask cluster available to the class:

In [ ]:
import coiled
cluster = coiled.Cluster(name="ds2-highcpu-binder", workspace="class-2025")
# cluster = coiled.Cluster(name="ds2-highmem-binder", workspace="class-2025")
client = cluster.get_client()

### Compute OHC

You now have everything you need to compute the 4-dimensional OHC variable and check its time evolution.

We recall that the Ocean Heat Content $OHC(x, y, z, t)$ is expressed in Joules an is given by:
$$OHC(x,y,z,t) = \rho \cdot C_p \cdot T \cdot \delta V$$

Again, you can let xarray to handle the dimensions broadcasting for you:

In [ ]:
%%time
rho = 1035 # kg/m3
Cp = -4186 # J/K/kg

ds['OHC'] = 
ds['OHC'].attrs = {'unit': 'J', 'long_name': 'Ocean Heat Content'}
ds['OHC']

### Compute timeseries for a specific layer

A classic way to visualise these timeseries is by integrating vertically only between the surface and 700m. This ensures a more reliable and consistent estimate throughout the time period, starting in 1950.

Moreover, global OHC timeseries are often given in $ZJ = 10^{21} J$ and as anomaly compared to the first timeseries value.

Let's now select the ocean layer 0-700 and compute the heat content:

In [ ]:
%%time
ohc700 = 
ohc700

It is important to note here that we did not yet computed anything, only the dimensions and coordinates where determined by xarray.

IT IS ONLY WHEN ARRAY VALUES ARE NEEDED THAT THE COMPUTATION REALY HAPPENS !

This is the case when doing a plot for instance. So to avoid to recompute `ohc700` every time we're plotting it, you can manually trigger the computation and make the result persistant like this:

In [ ]:
ohc700 = ohc700.compute().persist()

And finaly we can make the OHC 0-700m timeseries plot:

In [ ]:
origin = ohc700.isel(time=0)
unit = 1e21 # Zeta-joules

((ohc700 - origin)/unit).plot()
plt.title("OHC 0-700m timeseries from the EN4 dataset\n(in ZJ=10^{21}J and anomaly wrt %s)" % pd.to_datetime(origin['time'].data))
plt.grid()

## Local temperature trends

It is very interesting to be able to visualise a map of the local temperature trends in order to understand where is ocean warming the strongest.

To compute local trends, we will let xarray handle the latitude and longitude dimensions and simply works with xarray DataArrays.

Local trends can be computed as the ratio of the time/temperature covariance with time variance. 

So let's define the following sum of squares:

for time:
$$ss_{tt} = \Sigma_{i=1}^{N} (t_i - \bar{t})^2$$

for temperature:
$$ss_{\theta\theta} = \Sigma_{i=1}^{N} (\theta_i -\bar{\theta})^2$$

for time/temperature:
$$ss_{t\theta} = \Sigma_{i=1}^{N} (t_i - \bar{t})(\theta_i -\bar{\theta})$$

Then, the slope of the linear least square fit is simply given by:
$$a = \frac{ss_{t\theta}}{ss_tt}$$

If we further estimate the variance for the fit error as:
$$s^2 = \frac{ss_{\theta\theta} - a ss_{t\theta}}{N-2}$$

the standard error on the slope $a$ is given by:
$$SE(a) = \frac{s}{\sqrt{ss_{tt}}}$$

So, let's look at the latitude/longitude map of the 0-700 layer mean temperature trend.

To compute the 0-700m mean temperature, we need to weight the temperature variable by the grid thickness and to do so we can use the xarray `weighted` method:

In [ ]:
%%time
da = (ds['temperature'].where(ds['depth']<=700).weighted(ds['dZ'])).mean(dim=['depth'])
da

Let's now compute all the required sum of squares.

Note that for time, we need to convert the numpy datetime format to a more numerical value, like julian days.

In [ ]:
# For time:
ds['t'] = xr.DataArray(da['time'].to_pandas().index.to_julian_date().values, dims='time', name='juld')
ssxx = ((ds['t']-ds['t'].mean(dim='time'))**2).sum(dim='time')

# For temperature:
ssyy = 

# For time vs temperature:
ssxy = 

Finaly compute the slope of the linear least square fit:

In [ ]:
trend = ssxy / ssxx  # K / days
trend.compute().persist()

Let's now make a nice map of the temperature trend:

In [ ]:
fig, proj, ax = create_map()
# fig, proj, ax = create_map(extent=[-90, 0, 0, 80])

(trend*1e3*10).plot(transform=proj, ax=ax, levels=13, cbar_kwargs={'shrink': 0.8})
ax.add_feature(cfeature.LAND, facecolor=[0.7]*3, zorder=100)
ax.set_title("Local 0-700m temperature trend in mK/decade")

## Vertical structure of OHC

You have briefly looked at the timeseries and horizontal distribution of OHC and temperature.

You can further explore the other dimensions of the dataset.

In particular, you may want to look at zonal section across the Pacific or Atlantic oceans, or simply zonal means.

## Prediction

As it says, you may want to try to predict the evolution of OHC700, localy with T700

## CMIP6 dataset

Since you have access to CMIP6 simulations, you could also look at the OHC700/T700 evolution in climate simulations, trying to compute the projected values in 2100, or to isolate the human finger print of the historical period (piControl vs historical runs).

[More on climate model simulations data](https://www.wcrp-climate.org/wgcm-cmip/wgcm-cmip6)

[You can check at the CMIP6 practice notebook](https://github.com/obidam/ds2-2025/blob/main/practice/environment/Access-CMIP6-data.ipynb)

[Access to CMIP6 data](https://cloud.google.com/blog/products/data-analytics/new-climate-model-data-now-google-public-datasets)